## Introduction on HKJC horse racing data scrapper

> - Author: Shenghui
> - Date: 2023/01

Hi, before we further moving on analysis on horce-racing ranking, it is important to get our data first.

This notebook will introduce you how to use local functions to acquire data from hkjc's website with detailed steps.

## 0 Requirements

- selenium >= 4.6.1 
- pandas >= 1.4.2

## 1 Task: when to scrape?

The very first step is to tell our scrapper when to begin, and how long we should go?

Note that horse racing only opens at every wednesday in Sha Tin(ST), and every sunday in Happy Valley(HV) in Hong Kong, we can narrow our date of interest only to these two date of week(DoW).

`scrapper.datetool.get_span_of_DoW(...)` would be a powerful tool to nominate the expected date.

With some initial days, we can retrieve expected DoWs with:
- expected span: i.e. we want 30 wednesdays' date in string format before '2023/01/01';
- or date range: i.e. we want all wedesdays' date in string format between '2023/01/01' and '2022/12/01'.

For example:

In [3]:
# load local package
from horse.scrapper.datetool import get_span_of_DoW


""" Way 1. begin date with expected span """

BEGIN_DATE = '2023/01/19'   # setting: where we stand?
SPAN = 3                    # setting: how much weekdays to scrap?

list_of_date_1 = get_span_of_DoW(
    begin_date=BEGIN_DATE, span=SPAN 
    , date_of_interest='sunday')

print(list_of_date_1)


""" Way 2. begin date of range """

BEGIN_DATE = '2023/01/19'   # setting: where we stand?
END_DATE = '2022/12/01'     # setting: how much weekdays to scrap?

list_of_date_2 = get_span_of_DoW(
    begin_date=BEGIN_DATE, end_date=END_DATE 
    , date_of_interest='sunday')

print(list_of_date_2)

['2023/01/15', '2023/01/08', '2023/01/01']
['2023/01/15', '2023/01/08', '2023/01/01', '2022/12/25', '2022/12/18', '2022/12/11', '2022/12/04']


Formally:

In [1]:
# load local package
from horse.scrapper.datetool import get_span_of_DoW


BEGIN_DATE = '2023/01/19'
END_DATE = '2022/12/19'

# create a list to scrap in each course
horse_racing_dict = {
    'ST': get_span_of_DoW(BEGIN_DATE, end_date=END_DATE, date_of_interest='sunday')
    , 'HV': get_span_of_DoW(BEGIN_DATE, end_date=END_DATE, date_of_interest='wednesday')
}

# check elem
horse_racing_dict['ST'][:2], horse_racing_dict['HV'][:2]

(['2023/01/15', '2023/01/08'], ['2023/01/18', '2023/01/11'])

## 2 Init Webdriver & Scrape!

Given track and date of interest, we can then start scraping. For each combination, we plug params in `base_url`, and visit it.

With `scrapper.pagetool.get_performance_info(...)`, we can scrape data from the racing dashboard and store the information in `./data/XXX.txt`.


In [2]:
from selenium import webdriver
import re
import warnings
warnings.filterwarnings('ignore')

"""
https://racing.hkjc.com/racing/information/Chinese/Horse/Horse.aspx?HorseId=HK_2016_A098
"""


# load local package
from horse.scrapper.pagetool import get_performance_info

options=webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
chromedriver = r'./scrapper/chromedriver.exe'

# date format: 2022/12/18
# example: https://racing.hkjc.com/racing/information/Chinese/Racing/LocalResults.aspx?RaceDate=2022/12/18&Racecourse=ST&RaceNo=1

# hkjc_root = 'https://racing.hkjc.com'
base_url = 'https://racing.hkjc.com/racing/information/Chinese/Racing/LocalResults.aspx?RaceDate={0}&Racecourse={1}&RaceNo={2}'

driver = webdriver.Chrome(executable_path=chromedriver, chrome_options=options)

for race_course in horse_racing_dict:
    for race_date in horse_racing_dict[race_course]:
        for race_no in range(1,11):
            continue_flag = get_performance_info(driver, race_date, race_no, race_course, base_url)
            if not continue_flag:
                break

driver.close()

Scrapping: [Cost] - 7.74s. Target - https://racing.hkjc.com/racing/information/Chinese/Racing/LocalResults.aspx?RaceDate=2023/01/15&Racecourse=ST&RaceNo=1
Scrapping: [Cost] - 7.9s. Target - https://racing.hkjc.com/racing/information/Chinese/Racing/LocalResults.aspx?RaceDate=2023/01/15&Racecourse=ST&RaceNo=2
Scrapping: [Cost] - 10.81s. Target - https://racing.hkjc.com/racing/information/Chinese/Racing/LocalResults.aspx?RaceDate=2023/01/15&Racecourse=ST&RaceNo=3
Scrapping: [Cost] - 7.78s. Target - https://racing.hkjc.com/racing/information/Chinese/Racing/LocalResults.aspx?RaceDate=2023/01/15&Racecourse=ST&RaceNo=4
Scrapping: [Cost] - 10.46s. Target - https://racing.hkjc.com/racing/information/Chinese/Racing/LocalResults.aspx?RaceDate=2023/01/15&Racecourse=ST&RaceNo=5
Scrapping: [Cost] - 96.11s. Target - https://racing.hkjc.com/racing/information/Chinese/Racing/LocalResults.aspx?RaceDate=2023/01/15&Racecourse=ST&RaceNo=6
Scrapping: [Cost] - 7.78s. Target - https://racing.hkjc.com/racing/i

## 3 Cleanse the data

Since there're jockeys, trainers and horses attending multiple games, it is important to drop duplicated data before we analyze it.

we define `drop_duplicate(...)` to meet our need. 

In [25]:
def drop_duplicate(path, sep):
    from pandas import read_csv
    df = read_csv(path, sep=sep, encoding='utf-8')
    df_dedup = df.drop_duplicates()
    print(df.shape, df_dedup.shape)
    df_dedup.to_csv(path, sep=sep, index=False, encoding='utf-8')

drop_duplicate('./data/performance.txt', sep=':')
drop_duplicate('./data/url_horse.txt', sep=':')
drop_duplicate('./data/url_jockey.txt', sep=':')
drop_duplicate('./data/url_trainer.txt', sep=':')

(55532, 21) (55532, 21)
(4560, 2) (4560, 2)
(75, 2) (75, 2)
(73, 2) (73, 2)


## 4 Show details of raw data

In [26]:
from pandas import read_csv

df = read_csv('./data/performance.txt', sep=':', encoding='utf-8')
df.sample(5)

,Race Date,Race No.,Race Course,Distance,Field Going,Race Name,Course Type,Race Money,Pla.,Horse No.,...,Jockey,Trainer,Act. Wt.,Declare. Horse Wt.,Dr.,LBW,Running Position,Finish Time(min),Finish Time(sec),Win Odds
45597,2018/02/14,8,HV,第三班 - 1650米 - (80-60),好地,雛菊讓賽,草地 - B 賽道,"HK$ 1,310,000",3,5,...,莫雷拉,告東尼,126,1086,6,3/4,5 6 5 3,1,40.02,4.2
26143,2017/11/26,3,ST,第三班 - 1200米 - (80-60),好地,其士建築讓賽,全天候跑道,"HK$ 1,310,000",8,1,...,巴度,苗禮德,133,1091,2,5-1/4,9 7 8,1,09.75,9.6
20473,2019/05/05,10,ST,第三班 - 2000米 - (85-60),好地至黏地,施文信盃（讓賽）,草地 - B 賽道,"HK$ 1,380,000",5,9,...,潘明輝,沈集成,115,1181,5,3-3/4,10 10 10 11 5,2,03.80,5.7
54179,2015/05/20,5,HV,第四班 - 1200米 - (60-40),黏地,法國五月盃（讓賽）,草地 - C 賽道,"HK$ 760,000",6,4,...,羅理雅,高伯新,127,1137,12,6-3/4,11 11 6,1,12.35,21.0
12112,2021/09/22,2,HV,第四班 - 1650米 - (60-40),好地至快地,摩頓讓賽,草地 - C+3 賽道,"HK$ 1,040,000",8,8,...,賀銘年,葉楚航,119,1040,9,6,6 6 6 8,1,40.36,14.0


In [27]:
df = read_csv('./data/url_horse.txt', sep=':', encoding='utf-8')
df.sample(5)

,horse,url
4235,超奇妙(T057),/racing/information/Chinese/Horse/Horse.aspx?H...
2300,好醒神(C175),/racing/information/Chinese/Horse/Horse.aspx?H...
659,紅褲之王(E262),/racing/information/Chinese/Horse/Horse.aspx?H...
2165,吉金寶(C298),/racing/information/Chinese/Horse/Horse.aspx?H...
2174,盛唐文心(C390),/racing/information/Chinese/Horse/Horse.aspx?H...


In [28]:
df = read_csv('./data/url_jockey.txt', sep=':', encoding='utf-8')
df.sample(5)

,jockey,url
43,蔡明紹,/racing/information/Chinese/Jockey/JockeyProfi...
39,薛恩,/racing/information/Chinese/Jockey/JockeyProfi...
60,黎海榮,/racing/information/Chinese/Jockey/JockeyProfi...
63,馬昆,/racing/information/Chinese/Jockey/JockeyProfi...
57,蔡明紹,/racing/information/chinese/Jockey/JockeyProfi...


In [29]:
df = read_csv('./data/url_trainer.txt', sep=':', encoding='utf-8')
df.sample(5)

,trainer,url
23,姚本輝,/racing/information/chinese/Trainers/TrainerPr...
52,沈集成,/racing/information/chinese/Trainers/TrainerPr...
70,音無秀孝,/racing/information/Chinese/Trainers/TrainerPr...
10,伍鵬志,/racing/information/Chinese/Trainers/TrainerPr...
55,堀宣行,/racing/information/chinese/Trainers/TrainerPr...
